# KMEANS

### 18/05/20

## Importing Libraries

In [72]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import sys  
sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
from AdaBandFlt import *

%matplotlib tk

## Loading Data

In [73]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt' 
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [74]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

#xminnoise = int(np.round(11114*(fs/1000)))
#xmaxnoise = int(np.round(18511*(fs/1000)))

#noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

#xminspike = int(np.round(130826*(fs/1000)))
#xmaxspike = int(np.round(131699*(fs/1000)))

#burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

#begin_data = signal_filtered.iloc[:500000]

########################################################
y_ref = butter_bandpass_filter(df['El 15       '], lowcut, highcut, fs, order=6)

filtereddf_ref = pd.DataFrame(y_ref)
filtereddf_ref.index = df.index


In [75]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered
signal_filtered_ref = filtereddf_ref.iloc[:,0] #Entire recording filtered by bandpass, for reference electrode

## Detecting and Aligning the Spikes

### Setting Up Thresholds adapted to Noise Levels

In [76]:
fs = 25000

noise_levels = init_noise_levels(signal_filtered_ref, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)


In [77]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

### Detecting Spikes

In [78]:
spike_info = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.0005, 
                           noise_window_size = 0.01,
                           threshold_factor = 4,
                           maxseparation = 0.0015)

#spike_info.columns = ['indice_max','indice_min','indice_depass_positif','indice_depass_negatif', 'indice_1er_depass','indice_zero_central','i_max-i_min','Delta_amplitudes']
spike_info

,indice_max,indice_min,indice_depass_positif,indice_depass_negatif,indice_1er_depass,indice_zero_central,i_max-i_min,Delta_amplitudes
0,1142,1149,1141,1148,1141,1145,-7,9.648898
1,1163,1171,1159,1170,1159,1167,-8,14.314660
2,2091,2098,2089,2097,2089,2094,-7,9.816041
3,2253,2241,2251,2239,2239,2246,12,11.429386
4,3207,3198,3207,3197,3197,3202,9,8.312351
...,...,...,...,...,...,...,...,...
2064,518256,518275,518255,518274,518255,518260,-19,9.394551
2065,518463,518446,518462,518446,518446,518457,17,8.319177
2066,519022,519015,519021,519015,519015,519018,7,9.403910
2067,519356,519366,519354,519365,519354,519361,-10,9.523136


### Recording the spikes

In [79]:
spike_data = record_spikes(signal, fs, spike_info,
                           'indice_zero_central',
                           t_before = 0.0015, 
                           t_after = 0.0015)


(2070, 77)


In [80]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                                           'indice_zero_central',
                                           t_before = 0.0015,
                                           t_after = 0.0015)

### Plotting the spikes

In [81]:
#signal.plot(color = 'blue')
#spike_data_oneline.plot(color = 'red')
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

No handles with labels found to put in legend.


In [82]:
print_spikes(spike_data,
             t_before_alignement = 0.0015,
             first_spike = 1,
             last_spike = 50,
             fs = 25000)

# Bilan PCA + KMEANS

## PCA and KMEANS on spikes

In [83]:
def PCA_and_KMEANS_spikes(spike_data, spike_info, nb_PCA_components=3, nb_clusters=3):
    
    ## PCA
    pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## KMEANS
    kmeans = KMeans(n_clusters=nb_clusters, random_state=0).fit(PCA_X)

    
    ## Ajout du label des clusters dans spike info
    spike_info['cluster_label'] = kmeans.labels_
    
    return PCA_X, kmeans, spike_info
    

## Plotting the PCA

In [84]:
## Fonction qui plot la PCA

def PCA_plot(PCA_X):
    
    fig = plt.figure(4,figsize=(4,3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    plt.cla()
  
    ax.scatter(PCA_X[:, 0], PCA_X[:, 1], PCA_X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])

    plt.show()

In [85]:
## Importance des premiers coefficients de la PCA selon les différents indices d'alignement

def print_pca_explained_variance_ratio(signal, fs, spike_infos):
    legend = ['indice_max',
     'indice_min',
     'indice_depass_positif',
     'indice_depass_negatif',
     'indice_1er_depass',
     'indice_zero_central']
    explained_variance_ratio = []
    
    for method in legend:
        spike_data = record_spikes(signal, fs, spike_infos,
                  method,
                  t_before = 0.001,
                  t_after = 0.002)
        pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
        pca = PCA(n_components=10)
        pca.fit(pca_data)
        X = pca.transform(pca_data)
        explained_variance_ratio.append(pca.explained_variance_ratio_)
        
    plt.figure()
    plt.plot(np.transpose(explained_variance_ratio))
    plt.legend(legend)

## Plotting the KMEANS

In [86]:
## Fonction qui plot le KMEANS

def print_clusters_3d(labels, PCA_X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    nb_clusters = max(labels) + 1

    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])

    for nb in range(nb_clusters):
        data = PCA_X[[x for x,y in b if y==nb],:]
        ax.scatter(data[:,0], data[:,1], data[:,2])
    data = PCA_X[[x for x,y in b if y==-1],:]
    ax.scatter(data[:,0], data[:,1], data[:,2], c='black')    
    
    ax.set_title('Nombre de cluster(s) :' + str(nb_clusters))
    plt.show()
    
#print_clusters_3d(kmeans.labels_, X)

## Plotting the spikes from the different clusters after KMEANS

In [87]:
## Fonction qui plot les spikes dans les clusters du KMEANS

def print_spikes_clusterized(spike_data,
                             labels,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000):
    
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 1
    else:
        nb_clusters_ = nb_clusters
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    #spike_data.iloc[:,first_spike:last_spike].plot()
    t_b = int(np.round(fs*(t_before_alignement)))
    y = (spike_data.iloc[:,0]-t_b)*1000/fs
        
    a = [i+1 for i in range(len(labels))]
    b = np.transpose([a,list(labels)])
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        data = spike_data.iloc[:,[x for x,y in b if y==nb]]
        m = len(data.values[0])
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster numero ' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        
    if (-1 in labels) == True:
        data = spike_data.iloc[:,[x for x,y in b if y==-1]]
        m = len(data.values[0])
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        
#print_spikes_clusterized(spike_data,
#                             dbscan.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
        

#print_spikes_clusterized(spike_data,
#                             kmeans.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)

## Printing the spikes from clusters on original signal

In [ ]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline(signal,
                                     updated_spike_infos,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    labels = updated_spike_infos['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 2
    else:
        nb_clusters_ = nb_clusters + 1
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    ghost_array = np.array(['NaN' for x in range(len(signal))])
    ghost_array = ghost_array.astype(float)
    spike_data_clusterized_oneline = []
    legend = ['Signald\'origine']
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == nb],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(signal.index, signal.values)
        for ghost in range(nb):
            axes.plot(signal.index, ghost_array)
        axes.plot(signal.index, spike_data_oneline)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster n°' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        legend.append('Cluster n°' + str(nb))
        
    if (-1 in labels) == True:
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == -1],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, spike_data_oneline)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
                
        legend.append('Cluster de bruit')
        
        axes = figure.add_subplot(nb_line, 2, nb+3)
    else:
        axes = figure.add_subplot(nb_line, 2, nb+2)
    
    axes.plot(signal.index, signal.values)
    axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
    axes.legend(legend)
    axes.set_xlabel('Time in ms')
    axes.set_title('Tous les clusters')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

## Histogram of Amplitudes and Time

In [89]:
## Histogramme de différences d'amplitude Peak-to-Peak des spikes

def histogram_spikes_amplitude(spike_info):
    plt.figure()
    plt.hist(spike_info['Delta_amplitudes'], bins='auto')
    plt.title("Histogram of Spike Peak-to-Peak Amplitude")
    plt.show()

In [90]:
## Histogramme de l'écartement temporel des spikes (imax-imin)

def histogram_spikes_time(spike_info):
    plt.figure()
    plt.hist(spike_info['i_max-i_min'], bins='auto')
    plt.title("Histogram of Spike i_max-i_min")
    plt.show()

In [91]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized(updated_spike_info):
    
    labels = updated_spike_info['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters):
        local_info = spike_info.loc[spike_info['cluster_label'] == nb]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster n°"+str(nb))
        plt.show()
    
    if(-1 in labels) == True:
        local_info = spike_info.loc[spike_info['cluster_label']==-1]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster of noise")
        plt.show()

In [92]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized_superposed(updated_spike_info):
    
    labels = updated_spike_info['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters,0,-1):
        local_info = spike_info.loc[spike_info['cluster_label'] == nb]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Superposition of histogram of Spike peak-to-peak of clusters")
        plt.show()
    
    if(-1 in labels) == True:
        local_info = spike_info.loc[spike_info['cluster_label']==-1]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.show()

# Tests des fonctions

In [93]:
# Function: PCA_and_KMEANS_spikes(spike_data, spike_info, nb_PCA_components=3, nb_clusters=3)

PCA_X, KMEANS, updated_spike_info = PCA_and_KMEANS_spikes(spike_data, spike_info, 5, 4)

In [94]:
# Function: PCA_plot(spike_data, nb_clusters=3)

PCA_plot(PCA_X)

In [95]:
# Function: print_clusters_3d(labels, PCA_X)

print_clusters_3d(KMEANS.labels_, PCA_X)

In [96]:
# Function: print_spikes_clusterized(spike_data,
#                             labels,
#                             t_before_alignement = 0,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
                        
print_spikes_clusterized(spike_data,
                             KMEANS.labels_,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000)                        

In [97]:
# Function: print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

print_spikes_clusterized_oneline(signal,
                                 updated_spike_info,
                                 align_method = 'indice_zero_central',
                                 t_before = 0.0015,
                                 t_after = 0.0015,
                                 fs = 25000,)

In [98]:
# Function: histogram_spikes_amplitude(spike_info)

histogram_spikes_amplitude(spike_info)

In [99]:
# Function: histogram_spikes_time(spike_info)

histogram_spikes_time(spike_info)

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [100]:
# Function: histogram_amplitude_clusterized(spike_info)

histogram_amplitude_clusterized(updated_spike_info)

In [111]:
# Function: histogram_amplitude_clusterized_superposed(updated_spike_info)

histogram_amplitude_clusterized_superposed(updated_spike_info)